In [50]:
import numpy as np
import tensorflow as tf
import operator

In [51]:
# download csv file and translate to list
from six.moves.urllib.request import urlretrieve
import csv

url = 'https://s3-ap-northeast-1.amazonaws.com/furbo-mixpenal/kaggle_data/'
trainFile, _ = urlretrieve(url + 'train.csv')
testFile, _ = urlretrieve(url + 'test.csv')

trainData = list()
with open(trainFile, 'rt') as csvfile:
    spamreader = csv.DictReader(csvfile)
    for row in spamreader:
        trainData.append(row)
        
testData = list()
with open(testFile, 'rt') as csvfile:
    spamreader = csv.DictReader(csvfile)
    for row in spamreader:
        testData.append(row)

In [52]:
print(trainData[0])
print(trainData[0].get('PassengerId'))

print(testData[0])
print(testData[0].get('PassengerId'))

OrderedDict([('PassengerId', '1'), ('Survived', '0'), ('Pclass', '3'), ('Name', 'Braund, Mr. Owen Harris'), ('Sex', 'male'), ('Age', '22'), ('SibSp', '1'), ('Parch', '0'), ('Ticket', 'A/5 21171'), ('Fare', '7.25'), ('Cabin', ''), ('Embarked', 'S')])
1
OrderedDict([('PassengerId', '892'), ('Pclass', '3'), ('Name', 'Kelly, Mr. James'), ('Sex', 'male'), ('Age', '34.5'), ('SibSp', '0'), ('Parch', '0'), ('Ticket', '330911'), ('Fare', '7.8292'), ('Cabin', ''), ('Embarked', 'Q')])
892


In [44]:
# create index map for attribute => direct create by data order, train result: 0.692993
# map_list = ['Sex', 'Embarked']
# map_dict = dict()
# for index in map_list:
#     map_dict[index] = dict()
# for data in trainData:
#     for index in map_list:
#         if data.get(index) not in map_dict.get(index):
#             map_dict.get(index)[data.get(index)] = len(map_dict.get(index))
# print(map_dict)

{'Sex': {'male': 0, 'female': 1}, 'Embarked': {'S': 0, 'C': 1, 'Q': 2, '': 3}}


In [53]:
# check survived rate for each attribute
map_list = ['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
map_dict = dict()
for index in map_list:
    map_dict[index] = dict()
for data in trainData:
    for index in map_list:
        sub_dict = map_dict.get(index)
        if data.get(index) not in sub_dict:
            sub_dict[data.get(index)] = {
                'Survived': 0,
                'Dead': 0
            }
        if data.get('Survived') == '1':
            sub_dict.get(data.get(index))['Survived'] += 1
        else:
            sub_dict.get(data.get(index))['Dead'] += 1

for key in map_dict:
    sub_dict = map_dict.get(key)
    for index in sub_dict:
        sub_dict.get(index)['rate'] = sub_dict.get(index)['Survived'] / (sub_dict.get(index)['Survived'] + sub_dict.get(index)['Dead'])

orderDict = dict()
for index in map_list:
    subDict = map_dict.get(index)
    sortDict = sorted(subDict.items(), key=operator.itemgetter(0))
    orderDict[index] = [tup[0] for tup in sortDict]
# print(orderDict)

mapDict = dict()
for index in map_list:
    mapDict[index] = { key: i for i,key in enumerate(orderDict[index]) }
print(mapDict)

{'Pclass': {'1': 0, '2': 1, '3': 2}, 'Sex': {'female': 0, 'male': 1}, 'Age': {'': 0, '0.42': 1, '0.67': 2, '0.75': 3, '0.83': 4, '0.92': 5, '1': 6, '10': 7, '11': 8, '12': 9, '13': 10, '14': 11, '14.5': 12, '15': 13, '16': 14, '17': 15, '18': 16, '19': 17, '2': 18, '20': 19, '20.5': 20, '21': 21, '22': 22, '23': 23, '23.5': 24, '24': 25, '24.5': 26, '25': 27, '26': 28, '27': 29, '28': 30, '28.5': 31, '29': 32, '3': 33, '30': 34, '30.5': 35, '31': 36, '32': 37, '32.5': 38, '33': 39, '34': 40, '34.5': 41, '35': 42, '36': 43, '36.5': 44, '37': 45, '38': 46, '39': 47, '4': 48, '40': 49, '40.5': 50, '41': 51, '42': 52, '43': 53, '44': 54, '45': 55, '45.5': 56, '46': 57, '47': 58, '48': 59, '49': 60, '5': 61, '50': 62, '51': 63, '52': 64, '53': 65, '54': 66, '55': 67, '55.5': 68, '56': 69, '57': 70, '58': 71, '59': 72, '6': 73, '60': 74, '61': 75, '62': 76, '63': 77, '64': 78, '65': 79, '66': 80, '7': 81, '70': 82, '70.5': 83, '71': 84, '74': 85, '8': 86, '80': 87, '9': 88}, 'SibSp': {'0': 0

In [66]:
# translate data to 2-d matrix dataset AND 1-d label
train_data = list()
train_label = list()
index_list = ['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
# mapping_list = ['Sex','Ticket','Fare','Cabin','Embarked']
mapping_list = ['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
for data in trainData:
    data_list = list()
    for index in index_list:
        if index in mapping_list:
            data_list.append(mapDict.get(index)[data.get(index)]/len(mapDict.get(index)))
        else:
            if index is 'Age' and data.get(index) is '':
                data_list.append(0)
            else:
                data_list.append(data.get(index))
    train_data.append(data_list)
    if data.get('Survived') is '0':
        train_label.append([1,0])
    else:
        train_label.append([0,1])
print(train_data[0:3])
print(train_label[0:3])
print(test_data[0:3])


test_data = list()
index_list = ['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
for data in testData:
    data_list = list()
    for index in index_list:
        if index in mapping_list:
            if data.get(index) in mapDict.get(index):
                data_list.append(mapDict.get(index)[data.get(index)]/len(mapDict.get(index)))
            else:
#                 data_list.append(len(mapDict.get(index)))
#                 data_list.append(-1/len(mapDict.get(index)))
#                 print(data.get('PassengerId'),index,data.get(index))
                data_list.append(100)
        else:
            if index is 'Age' and data.get(index) is '':
                data_list.append(0)
            else:
                data_list.append(data.get(index))
    test_data.append(data_list)

[['1', 0.6666666666666666, 0.5, 0.24719101123595505, 0.14285714285714285, 0.0, 0.7016129032258065, 0.75], ['2', 0.0, 0.0, 0.5168539325842697, 0.14285714285714285, 0.0, 0.8024193548387096, 0.25], ['3', 0.6666666666666666, 0.0, 0.3146067415730337, 0.0, 0.0, 0.7943548387096774, 0.75]]
[[1, 0], [0, 1], [0, 1]]
[['892', 0.6666666666666666, 0.5, 0.4606741573033708, 0.0, 0.0, 0.7701612903225806, 0.5], ['893', 0.6666666666666666, 0.0, 0.651685393258427, 0.14285714285714285, 0.0, 1, 0.75], ['894', 0.3333333333333333, 0.5, 0.8539325842696629, 0.0, 0.0, 1, 0.5]]


In [60]:
print(len(train_data))
print(len(train_data[0]))
print(len(train_label))
print(len(train_label[0]))
print(len(test_data))
print(len(test_data[0]))

891
8
891
2
418
8


In [64]:
# training model
learning_rate = 0.001
x = tf.placeholder(tf.float32, [None, 8])
W = tf.Variable(tf.zeros([8, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))

0.693016
0.693012
0.693007
0.693003
0.692998


In [67]:
# get test result
# print(test_data)
print(sess.run(y, feed_dict={x: test_data}))

[[  1.00000000e+00   6.82777173e-21]
 [  1.00000000e+00   6.43488032e-21]
 [  1.00000000e+00   6.10845619e-21]
 [  1.00000000e+00   5.84045896e-21]
 [  1.00000000e+00   5.54488649e-21]
 [  1.00000000e+00   5.26302712e-21]
 [  1.00000000e+00   4.99648594e-21]
 [  1.00000000e+00   4.74293731e-21]
 [  1.00000000e+00   4.50259853e-21]
 [  1.00000000e+00   4.27388390e-21]
 [  1.00000000e+00   4.05700425e-21]
 [  1.00000000e+00   3.85151166e-21]
 [  1.00000000e+00   3.62991127e-21]
 [  1.00000000e+00   3.47056308e-21]
 [  1.00000000e+00   3.29477468e-21]
 [  1.00000000e+00   3.12777104e-21]
 [  1.00000000e+00   2.96893762e-21]
 [  1.00000000e+00   2.81821323e-21]
 [  1.00000000e+00   2.67526305e-21]
 [  1.00000000e+00   2.53967995e-21]
 [  1.00000000e+00   2.41087646e-21]
 [  1.00000000e+00   2.27180046e-21]
 [  1.00000000e+00   2.15687890e-21]
 [  1.00000000e+00   2.06233656e-21]
 [  1.00000000e+00   1.95784664e-21]
 [  1.00000000e+00   1.85827518e-21]
 [  1.00000000e+00   1.76426558e-21]
 

In [39]:
# training model by different optimizer
learning_rate = 0.001
x = tf.placeholder(tf.float32, [None, 8])
W = tf.Variable(tf.zeros([8, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
print('GradientDescentOptimizer')
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))


print('AdamOptimizer')
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))
    


print('RMSPropOptimizer')
train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))
    


print('ProximalAdagradOptimizer')
train_step = tf.train.ProximalAdagradOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))

GradientDescentOptimizer
0.693016
0.693012
0.693007
0.693003
0.692998
AdamOptimizer
0.671513
0.675268
0.678405
0.680121
0.680957
RMSPropOptimizer
0.68512
0.683858
0.681986
0.679101
0.674895
ProximalAdagradOptimizer
0.671667
0.671634
0.671616
0.671602
0.67159


In [68]:
# use survived rate as index
map_list = ['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
map_dict = dict()
for index in map_list:
    map_dict[index] = dict()
for data in trainData:
    for index in map_list:
        sub_dict = map_dict.get(index)
        if data.get(index) not in sub_dict:
            sub_dict[data.get(index)] = {
                'Survived': 0,
                'Dead': 0
            }
        if data.get('Survived') == '1':
            sub_dict.get(data.get(index))['Survived'] += 1
        else:
            sub_dict.get(data.get(index))['Dead'] += 1

for key in map_dict:
    sub_dict = map_dict.get(key)
    for index in sub_dict:
        sub_dict.get(index)['rate'] = sub_dict.get(index)['Survived'] / (sub_dict.get(index)['Survived'] + sub_dict.get(index)['Dead'])

In [69]:
# translate data to 2-d matrix dataset AND 1-d label
train_data = list()
train_label = list()
index_list = ['PassengerId','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
mapping_list = ['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
for data in trainData:
    data_list = list()
    for index in index_list:
        if index in mapping_list:
            rate = map_dict.get(index).get(data.get(index))['rate']
            data_list.append(rate)
        else:
            data_list.append(data.get(index))
    train_data.append(data_list)
    if data.get('Survived') is '0':
        train_label.append([1,0])
    else:
        train_label.append([0,1])
print(train_data[0:3])
print(train_label[0:3])


test_data = list()
for data in testData:
    data_list = list()
    for index in index_list:
        if index in mapping_list:
            if data.get(index) in mapDict.get(index):
                rate = map_dict.get(index).get(data.get(index))['rate']
                data_list.append(rate)
            else:
                data_list.append(0.5)
        else:
            data_list.append(data.get(index))
    test_data.append(data_list)

[['1', 0.24236252545824846, 0.18890814558058924, 0.4074074074074074, 0.5358851674641149, 0.34365781710914456, 0.0, 0.07692307692307693, 0.29985443959243085, 0.33695652173913043], ['2', 0.6296296296296297, 0.7420382165605095, 0.45454545454545453, 0.5358851674641149, 0.34365781710914456, 1.0, 1.0, 1.0, 0.5535714285714286], ['3', 0.24236252545824846, 0.7420382165605095, 0.3333333333333333, 0.34539473684210525, 0.34365781710914456, 1.0, 0.4444444444444444, 0.29985443959243085, 0.33695652173913043]]
[[1, 0], [0, 1], [0, 1]]


In [73]:
# training model by different optimizer
learning_rate = 0.001
x = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([10, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
print('GradientDescentOptimizer')
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train_step, feed_dict = {x: train_data, y_: train_label})
    print(sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))

GradientDescentOptimizer
0.693015
0.693011
0.693006
0.693001
0.692997


In [71]:
print(len(train_data))
print(len(train_data[0]))
print(len(train_label))
print(len(train_label[0]))
print(len(test_data))
print(len(test_data[0]))

891
10
891
2
418
10


In [74]:
print(sess.run(y, feed_dict={x: test_data}))

[[  1.00000000e+00   6.83108045e-21]
 [  1.00000000e+00   6.48473128e-21]
 [  1.00000000e+00   6.15566082e-21]
 [  1.00000000e+00   5.84317800e-21]
 [  1.00000000e+00   5.54793308e-21]
 [  1.00000000e+00   5.26541657e-21]
 [  1.00000000e+00   4.99846907e-21]
 [  1.00000000e+00   4.74568825e-21]
 [  1.00000000e+00   4.50435104e-21]
 [  1.00000000e+00   4.27528623e-21]
 [  1.00000000e+00   4.05879957e-21]
 [  1.00000000e+00   3.85311352e-21]
 [  1.00000000e+00   3.65775309e-21]
 [  1.00000000e+00   3.47211243e-21]
 [  1.00000000e+00   3.29556672e-21]
 [  1.00000000e+00   3.12847503e-21]
 [  1.00000000e+00   2.97017233e-21]
 [  1.00000000e+00   2.81939584e-21]
 [  1.00000000e+00   2.67650827e-21]
 [  1.00000000e+00   2.54069757e-21]
 [  1.00000000e+00   2.41217378e-21]
 [  1.00000000e+00   2.28948730e-21]
 [  1.00000000e+00   2.17342251e-21]
 [  1.00000000e+00   2.06279276e-21]
 [  1.00000000e+00   1.95882528e-21]
 [  1.00000000e+00   1.85890627e-21]
 [  1.00000000e+00   1.76460223e-21]
 